# Модель предсказания UpLift для компании "Лента"

In [1]:
! pip install CatBoost
! pip install LightGBM

In [2]:
! pip install -U pandasql

^C
Traceback (most recent call last):
  File "<frozen importlib._bootstrap_external>", line 88, in _path_is_mode_type
  File "<frozen importlib._bootstrap_external>", line 82, in _path_stat
FileNotFoundError: [Errno 2] No such file or directory: '/anaconda/envs/azureml_py36_pytorch/lib/python3.6/site-packages/google/__init__.so'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda/envs/azureml_py36_pytorch/bin/pip", line 7, in <module>
    from pip._internal.cli.main import main
  File "/anaconda/envs/azureml_py36_pytorch/lib/python3.6/site-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/anaconda/envs/azureml_py36_pytorch/lib/python3.6/site-packages/pip/_internal/cli/autocompletion.py", line 9, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/anaconda/envs/azureml_py36_pytorch/lib/python3.6/site-packa

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
#import xgboost as xgb
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy import stats as st
import matplotlib.pyplot as plt
#import seaborn as sns
pd.options.display.max_columns = None
pd.set_option('display.float_format', lambda x: '%.3f' % x)
np.set_printoptions(precision=3,suppress=True)
import warnings
warnings.simplefilter("ignore")
from sklearn.preprocessing import StandardScaler
pd.options.mode.chained_assignment = None
import time
#from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit
import datetime

In [4]:
#import pandasql as ps

## Подготовка данных

In [6]:
checks = pd.read_csv('https://palitrastorage.blob.core.windows.net/data/20210518_checks.csv')

In [7]:
hierarchy = pd.read_csv('https://palitrastorage.blob.core.windows.net/data/20210518_hierarchy.csv')

In [8]:
uplift = pd.read_csv('https://palitrastorage.blob.core.windows.net/data/20210518_uplift.csv')

In [9]:
offers = pd.read_csv('https://palitrastorage.blob.core.windows.net/data/20210521_offers.csv')

Создадим функцию для перевода данных из формата int в формат даты.

In [10]:
def datification(weird_date):
    new_date = datetime.datetime.strptime(weird_date, '%Y%m%d')
    return new_date

In [11]:
offers['start_date'] = offers['start_date'].astype(str)
offers['start_date'] = offers['start_date'].apply(datification)
offers['end_date'] = offers['end_date'].astype(str)
offers['end_date'] = offers['end_date'].apply(datification)

In [12]:
offers.head()

,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group
0,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train
1,Facades,F2,e87788b585ead792a4d40f7de18f2f76,2020-01-02,2020-01-08,train
2,Facades,F9,5ae92b5e465d630285677e80b43fe5f7,2020-01-16,2020-01-22,train
3,Facades,F45,0790b472112a765339f0e5c3f66816ee,2020-03-19,2020-03-25,train
4,Facades,F191,e87788b585ead792a4d40f7de18f2f76,2020-11-12,2020-11-18,test


In [13]:
checks['day'] = checks['day'].astype(str)
checks['day'] = checks['day'].apply(datification)

In [14]:
checks.head()

,client_id,day,shop_id,check_id,time,sku,promo_id,check_pos,num_sales,supplier_price,selling_price,region_name
0,d9da50b77962ad6401d0527db5b65b04,2019-12-30,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,b17c870027bb4a22e3aedb971bc00def,b3deb0286313f0b888c0eac49580cc23,1,5.000,133.400,249.450,152f1b77a32508570e2745daf9ce7aec
1,d9da50b77962ad6401d0527db5b65b04,2019-12-30,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,b9ef00e971d0fb459a765708efba70c0,680a96609351230ed00e1d1090371cbc,2,1.000,96.770,118.990,152f1b77a32508570e2745daf9ce7aec
2,d9da50b77962ad6401d0527db5b65b04,2019-12-30,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,b75147feae4cf8ec07fb5f2cef855def,7215ee9c7d9dc229d2921a40e899ec5f,5,2.000,2.340,7.580,152f1b77a32508570e2745daf9ce7aec
3,d9da50b77962ad6401d0527db5b65b04,2019-12-30,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,b25e0d42165d246d23cbf4d3efcdf583,7215ee9c7d9dc229d2921a40e899ec5f,4,1.000,23.590,39.990,152f1b77a32508570e2745daf9ce7aec
4,d9da50b77962ad6401d0527db5b65b04,2019-12-30,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,a537d2eef30d47f9b11d655421ca93fb,05c93d4fa9bdbe70b5d2204ce9548ca9,3,1.000,24.020,39.990,152f1b77a32508570e2745daf9ce7aec


In [15]:
checks['selling_price_per_unit'] = checks['selling_price'] / checks['num_sales']

In [16]:
checks['supplier_price_per_unit'] = checks['supplier_price'] / checks['num_sales']

In [17]:
checks['promo_id'].value_counts().sort_values(ascending = False)

7215ee9c7d9dc229d2921a40e899ec5f    10881653
c4c67f412855bab90c566139cb2b3d83      182037
47b5739208ae99fe23f06fedb3ee8ea7       74030
d9ca24d75e5cecc5a01c452cc8b8288e       72601
7071bf06286a55deac8fa9619ae3517a       71467
                                      ...   
208dd3a9effe867436c8073731460fec           1
e92c071c4c018eaa749164c97af830bc           1
47d1526e2a38b49849b91103b22e99a2           1
81af02592dca9c223a9018542c0f0266           1
efe5ec3c83e39bf721c4bb4643709046           1
Name: promo_id, Length: 176939, dtype: int64

Больше всего товаров продаётся без акции. Заменим это значение promo_id на no_offers

In [18]:
checks['promo_id'] = checks['promo_id'].mask(checks['promo_id'] == '7215ee9c7d9dc229d2921a40e899ec5f', 'no_offers')

In [19]:
checks['promo_id'].value_counts().sort_values(ascending = False)

no_offers                           10881653
c4c67f412855bab90c566139cb2b3d83      182037
47b5739208ae99fe23f06fedb3ee8ea7       74030
d9ca24d75e5cecc5a01c452cc8b8288e       72601
7071bf06286a55deac8fa9619ae3517a       71467
                                      ...   
2d3fddf150a3fd7a3c4dadc2a40f5d56           1
946e4fb4d6a831ea9b4c2952a2afc0e7           1
bd2ce5bfd1ffb532942f5567e0d1ecda           1
8e9e39edb5d466808349a2aa943a13b4           1
a77f1c9089d4c3db2556b5d12dd3a6f7           1
Name: promo_id, Length: 176939, dtype: int64

Добавим к таблице checks данные по иерархии товаров.

In [20]:
checks_merged1 = checks.merge(hierarchy, how='outer', on = ['sku'])

In [21]:
offers['duration'] = offers['end_date'] - offers['start_date'] + pd.to_timedelta('1 day')

Для определения предакционного периода введём дату pre_start_datepre_start_date

In [22]:
offers['pre_start_date'] = offers['start_date'] - offers['duration']

In [23]:
checks_with_offers = checks_merged1.merge(offers, how='outer', on = ['sku'])

In [24]:
checks_with_offers_train = checks_with_offers.query('train_test_group in ("train")')

In [25]:
checks_with_offers_train

,client_id,day,shop_id,check_id,time,sku,promo_id,check_pos,num_sales,supplier_price,selling_price,region_name,selling_price_per_unit,supplier_price_per_unit,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4,Promo_type,Offer_ID,start_date,end_date,train_test_group,duration,pre_start_date
265,d9da50b77962ad6401d0527db5b65b04,2019-12-30,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,b9ef00e971d0fb459a765708efba70c0,680a96609351230ed00e1d1090371cbc,2,1.000,96.770,118.990,152f1b77a32508570e2745daf9ce7aec,118.990,96.770,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,287,2020-03-19,2020-04-01,train,14 days,2020-03-05
266,d9da50b77962ad6401d0527db5b65b04,2019-12-30,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,b9ef00e971d0fb459a765708efba70c0,680a96609351230ed00e1d1090371cbc,2,1.000,96.770,118.990,152f1b77a32508570e2745daf9ce7aec,118.990,96.770,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,369,2020-06-25,2020-07-08,train,14 days,2020-06-11
267,4d834f7ca2a047dd528cd67437612e03,2019-12-30,7d7c45b9a935cf9d845fc75679a41559,4b00d8b990484fde1f7db8c444790d26,15:58:46,b9ef00e971d0fb459a765708efba70c0,680a96609351230ed00e1d1090371cbc,2,1.000,96.770,118.990,7e35e74e610188414ad24235dd787c78,118.990,96.770,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,287,2020-03-19,2020-04-01,train,14 days,2020-03-05
268,4d834f7ca2a047dd528cd67437612e03,2019-12-30,7d7c45b9a935cf9d845fc75679a41559,4b00d8b990484fde1f7db8c444790d26,15:58:46,b9ef00e971d0fb459a765708efba70c0,680a96609351230ed00e1d1090371cbc,2,1.000,96.770,118.990,7e35e74e610188414ad24235dd787c78,118.990,96.770,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,369,2020-06-25,2020-07-08,train,14 days,2020-06-11
269,e21fddf3d068908dddd90810815f0004,2019-12-30,4bc8f1a601aee89546deeb66b9625008,a00f18d0ee1159d88a87f92b027febb1,11:12:14,b9ef00e971d0fb459a765708efba70c0,680a96609351230ed00e1d1090371cbc,1,1.000,96.770,118.990,7e35e74e610188414ad24235dd787c78,118.990,96.770,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,287,2020-03-19,2020-04-01,train,14 days,2020-03-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34334925,ab7537def14ed7ff7f0843b44b9d636e,2019-12-03,7f1a65908b05238c21c134c764c89e28,8ff909ceb0d997b0f8a4566543947531,13:43:06,5a7611f54e2ba26fe06f3f3c12ef3b02,ae8916fb97aa0dc9d3733d0b2cd6de51,8,1.000,14.400,14.890,152f1b77a32508570e2745daf9ce7aec,14.890,14.400,cc9284597026dcc021735c0970ca0c93,a1d0c6e83f027327d8461063f4ac58a6,7d265aa7147bd3913fb84c7963a209d1,65ce9a05680f7ff3339a68cc1eac2330,Billboards,B47,2020-06-11,2020-06-24,train,14 days,2020-05-28
34334926,ab7537def14ed7ff7f0843b44b9d636e,2019-12-03,7f1a65908b05238c21c134c764c89e28,8ff909ceb0d997b0f8a4566543947531,13:43:06,5a7611f54e2ba26fe06f3f3c12ef3b02,ae8916fb97aa0dc9d3733d0b2cd6de51,8,1.000,14.400,14.890,152f1b77a32508570e2745daf9ce7aec,14.890,14.400,cc9284597026dcc021735c0970ca0c93,a1d0c6e83f027327d8461063f4ac58a6,7d265aa7147bd3913fb84c7963a209d1,65ce9a05680f7ff3339a68cc1eac2330,Biweekly,68,2020-03-05,2020-03-18,train,14 days,2020-02-20
34334986,ca625b6c2339fbaa0874c8718b37a311,2020-08-11,8c5781246fcb0296e1f11cd4e2eee68d,ea9b172f9c4a034ac362042467f56689,12:47:24,8964706878d0b9b204714280296925d7,3321c53a192cfa09b6ac470996f05126,2,1.000,108.630,199.000,7e35e74e610188414ad24235dd787c78,199.000,108.630,44a7b825383402ed937a15fb8a5bb12a,9a1158154dfa42caddbd0694a4e9bdc8,f50c7035e532c49a0f6993d988e2e843,c134be80fec34c5b05f7464154900af8,Biweekly,368,2020-0

checks_with_offers_train.to_csv('checks_with_offers_train.csv')

In [27]:
checks_with_offers_action = checks_with_offers_train.query('day <= end_date and day >= start_date')

In [28]:
checks_with_offers_pred_action = checks_with_offers_train.query('day < start_date and day >= pre_start_date')

checks_with_offers_action.to_csv('checks_with_offers_action.csv')

In [ ]:
checks_with_offers_pred_action.to_csv('checks_with_offers_pred_action.csv')

In [30]:
checks_with_offers_action

,client_id,day,shop_id,check_id,time,sku,promo_id,check_pos,num_sales,supplier_price,selling_price,region_name,selling_price_per_unit,supplier_price_per_unit,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4,Promo_type,Offer_ID,start_date,end_date,train_test_group,duration,pre_start_date
1479,f5883bc2315398454cef231404da81ee,2020-03-27,d39934ce111a864abf40391f3da9cdf5,6b4d6d3a10deb5788eafca73c0e795fd,13:01:11,b9ef00e971d0fb459a765708efba70c0,1de8ed0a82fc39796d11da89ee86d607,46,1.000,96.770,114.990,152f1b77a32508570e2745daf9ce7aec,114.990,96.770,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,287,2020-03-19,2020-04-01,train,14 days,2020-03-05
1481,62823feeae170ce71a3787c203d8a718,2020-03-27,d39934ce111a864abf40391f3da9cdf5,4cec157ad32ea17bf0a3e4178019b990,15:15:53,b9ef00e971d0fb459a765708efba70c0,1de8ed0a82fc39796d11da89ee86d607,1,1.000,96.770,114.990,152f1b77a32508570e2745daf9ce7aec,114.990,96.770,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,287,2020-03-19,2020-04-01,train,14 days,2020-03-05
1483,073aa4b0edc4617c62c302aaf791a7cb,2020-03-27,d39934ce111a864abf40391f3da9cdf5,502843a48b0fc49f4acc134a7f0c03cd,08:00:35,b9ef00e971d0fb459a765708efba70c0,1de8ed0a82fc39796d11da89ee86d607,14,1.000,96.770,114.990,152f1b77a32508570e2745daf9ce7aec,114.990,96.770,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,287,2020-03-19,2020-04-01,train,14 days,2020-03-05
1485,9925f967444c9cee55518b25ee7a116a,2020-03-27,d39934ce111a864abf40391f3da9cdf5,d973669af03fca24d481b7b28d0256e9,20:51:32,b9ef00e971d0fb459a765708efba70c0,1de8ed0a82fc39796d11da89ee86d607,24,1.000,96.770,114.990,152f1b77a32508570e2745daf9ce7aec,114.990,96.770,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,287,2020-03-19,2020-04-01,train,14 days,2020-03-05
1487,cd6e626e2303c8c34c2b411d974a066a,2020-03-27,d39934ce111a864abf40391f3da9cdf5,3ba293abef2c1328081ccb6bc88524df,16:12:32,b9ef00e971d0fb459a765708efba70c0,1de8ed0a82fc39796d11da89ee86d607,1,1.000,96.770,114.990,152f1b77a32508570e2745daf9ce7aec,114.990,96.770,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,287,2020-03-19,2020-04-01,train,14 days,2020-03-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34328625,8ad501db6e0f885585b2e0dfa0e68d13,2020-08-19,d33f67c5652acb3fac99b9dc16bea0c1,99281b4fedff4c7d1f393ab80736b90d,11:20:05,80047755fdd5c6459ed2c4f734c190e4,792172eaeacc7979a5d0f50359d13322,12,1.000,206.140,298.430,7e35e74e610188414ad24235dd787c78,298.430,206.140,44a7b825383402ed937a15fb8a5bb12a,2838023a778dfaecdc212708f721b788,2c26f9a59b0ba61233e6fc0af8e47f14,e2c7a001428c33b71d08b54f254e0997,Seasonal,506,2020-07-28,2020-09-01,train,36 days,2020-06-22
34330360,fb704ef73f277eeff1154e7a8112941c,2020-06-28,29549a71a57f587d88209b9c1f1b7999,b5b9528029645c816ec2bc6858657855,15:21:39,9fe155a9e4f9402ba8728916a35a162e,883319be348e4a942d64134fbbbaf055,49,1.000,125.400,178.820,152f1b77a32508570e2745daf9ce7aec,178.820,125.400,44a7b825383402ed937a15fb8a5bb12a,9a1158154dfa42caddbd0694a4e9bdc8,f50c7035e532c49a0f6993d988e2e843,c134be80fec34c5b05f7464154900af8,Biweekly,368,2020-06-25,2020-07-08,train,14 days,2020-06-11
34330977,f40fefad0e55aa5c99a7e95553cdafef,2019-10-07,147768d3955e38c4e662c0a95d807abc,195b515ef0ea2719ebefd618b102d96c,13:26:34,275e38c99e9868037585564842839a8e,019043b5539be61b162515a5e0d9a5a7,11,1.000,455.930,798.630,152f1b77a32508570e2745daf9ce7aec,798.630,455.930,44a7b825383402ed937a15fb8a5bb12a,c0c7c76d30bd3dcaefc96f40275bdc0a,3202111cf90e7c816a472aaceb72b0df,d854234cf1f26b77b259a8511912040b,S

In [31]:
checks_with_offers_pred_action

,client_id,day,shop_id,check_id,time,sku,promo_id,check_pos,num_sales,supplier_price,selling_price,region_name,selling_price_per_unit,supplier_price_per_unit,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4,Promo_type,Offer_ID,start_date,end_date,train_test_group,duration,pre_start_date
965,1aa391faa9bac6134a0aa1aa6aa51fd8,2020-03-07,8c5781246fcb0296e1f11cd4e2eee68d,5c972464239bd4147d82e14ff74a93ef,17:38:37,b9ef00e971d0fb459a765708efba70c0,no_offers,12,1.000,100.640,188.890,7e35e74e610188414ad24235dd787c78,188.890,100.640,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,287,2020-03-19,2020-04-01,train,14 days,2020-03-05
967,677635a094c51b57726420f4ad6ee3e9,2020-03-07,adeb12111c95807557664efbf3b746ae,9ec1b02bc03e904bc031aadeafeb624c,08:03:23,b9ef00e971d0fb459a765708efba70c0,no_offers,13,1.000,104.310,199.390,7e35e74e610188414ad24235dd787c78,199.390,104.310,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,287,2020-03-19,2020-04-01,train,14 days,2020-03-05
969,06e50e2e042604a4c15f1d10d1d84911,2020-03-07,a410115ed452a9213424d92cacd2a083,a3bfb5ca27358cd89eefef669443906a,18:42:59,b9ef00e971d0fb459a765708efba70c0,no_offers,2,1.000,114.910,194.990,152f1b77a32508570e2745daf9ce7aec,194.990,114.910,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,287,2020-03-19,2020-04-01,train,14 days,2020-03-05
971,7275f3537f8a51ceeb5c66daac3786da,2020-03-07,1640f59af9fc45f93a72652de254d387,2bb71bd0be4750323de82b8b3f281f52,14:02:17,b9ef00e971d0fb459a765708efba70c0,no_offers,12,1.000,138.250,183.020,152f1b77a32508570e2745daf9ce7aec,183.020,138.250,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,287,2020-03-19,2020-04-01,train,14 days,2020-03-05
1179,e18655bda57e4041819e83c1c6993667,2020-03-07,f9d3742964e8d35a46da512cf46fbeb3,d7cf11065fa72ac1f963b77708fb30f0,19:52:23,b9ef00e971d0fb459a765708efba70c0,no_offers,12,1.000,96.770,195.090,7e35e74e610188414ad24235dd787c78,195.090,96.770,44a7b825383402ed937a15fb8a5bb12a,d645920e395fedad7bbbed0eca3fe2e0,6693a7aacb8a2e74c6f5fe91bc39864b,878b4140ebdb1b79f378d2cf0cc3245c,Biweekly,287,2020-03-19,2020-04-01,train,14 days,2020-03-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34313596,7553ca3847970838d2eb0c8e7482b574,2020-07-13,c0279f73075a52e1a7dea35065bc8c80,071796c257614814b2a3628df9c877ae,11:37:38,62237c588c8b170a1aa6388b5479ce60,no_offers,14,1.000,74.920,142.410,152f1b77a32508570e2745daf9ce7aec,142.410,74.920,44a7b825383402ed937a15fb8a5bb12a,9a1158154dfa42caddbd0694a4e9bdc8,c5d215777c229704a7862de577d40a73,31873776d238ef88ce7f69c799f7edfd,Seasonal,510,2020-07-28,2020-09-01,train,36 days,2020-06-22
34313952,bea55df34ca1452092fb3815f1a5f7e5,2020-06-07,898dc2c947cee718e4afd7dfcb2f1a09,92a98b1e61b9ad123d3acf3d9b393874,14:58:16,26f6fb076274b4416f995fe47547e803,a5529da46e1cea07199b3a4a906a8421,1,1.000,5561.190,13999.000,7e35e74e610188414ad24235dd787c78,13999.000,5561.190,44a7b825383402ed937a15fb8a5bb12a,66f041e16a60928b05a7e228a89c3799,1ce4fe042832e6bd7d06697a43055373,662e87c6f8d64f5261607715aff07b66,Seasonal,523,2020-06-29,2020-07-27,train,29 days,2020-05-31
34313957,c339acf196f01f1b4b1639b7fe7a7790,2020-06-08,e90b327c1ceca11f1a44a8bcf58af34c,87a576d6a59bd60459d87b672cadd045,14:33:24,26f6fb076274b4416f995fe47547e803,a5529da46e1cea07199b3a4a906a8421,1,1.000,6003.450,13999.000,7e35e74e610188414ad24235dd787c78,13999.000,6003.450,44a7b825383402ed937a15fb8a5bb12a,66f041e16a60928b05a7e228a89c3799,1ce4fe042832e6bd7d06697a43055373,662e87c6f8d64f5261607715aff07b66,Seasonal,523,2020-06-29,2020-07-27,train,29 days,2020-05-31
34318765,463a85bdd0ea19a697c62dba8141fb90,2020-06-24,95b09698fda1f

In [55]:
checks_with_offers_action_pivot_price = checks_with_offers_action.pivot_table(index=['Offer_ID', 'sku', 'region_name', 'Promo_type', 'start_date'], values=['selling_price_per_unit', 'supplier_price_per_unit'] , aggfunc=['mean'])
checks_with_offers_action_pivot_price.columns = ['mean_selling_price_per_unit_action', 'mean_supplier_price_per_unit_action']
checks_with_offers_action_pivot_price = checks_with_offers_action_pivot_price.reset_index()
checks_with_offers_action_pivot_price

,Offer_ID,sku,region_name,Promo_type,start_date,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action
0,10,172239f5f436f6b6661fdaa10fb47e9f,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,49.290,28.623
1,10,172239f5f436f6b6661fdaa10fb47e9f,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,48.832,28.504
2,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,49.355,28.832
3,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,49.372,28.428
4,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,49.346,31.137
...,...,...,...,...,...,...,...
14569,F98,ef7bf412117548cb80b3f3b33a12977d,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,115.438,110.175
14570,F99,5e3039a269ff005f2c76cea16c0cf783,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,71.952,48.791
14571,F99,5e3039a269ff005f2c76cea16c0cf783,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,81.594,38.961
14572,F99,6be0907d36fbbf5f7378f32c46f37943,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,29.943,22.317


In [56]:
checks_with_offers_action_pivot_amount = checks_with_offers_action.pivot_table(index=['Offer_ID', 'sku', 'region_name', 'Promo_type', 'start_date'], values=['num_sales', 'supplier_price', 'selling_price'] , aggfunc=['sum'])
checks_with_offers_action_pivot_amount.columns = ['num_sales_action', 'selling_price_action', 'supplier_price_action']
checks_with_offers_action_pivot_amount = checks_with_offers_action_pivot_amount.reset_index()
checks_with_offers_action_pivot_amount

,Offer_ID,sku,region_name,Promo_type,start_date,num_sales_action,selling_price_action,supplier_price_action
0,10,172239f5f436f6b6661fdaa10fb47e9f,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,95.000,4684.180,2715.750
1,10,172239f5f436f6b6661fdaa10fb47e9f,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,103.000,5036.900,2936.440
2,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,289.000,14265.200,8327.790
3,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,521.000,25722.260,14813.040
4,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,111.000,5477.810,3457.240
...,...,...,...,...,...,...,...,...
14569,F98,ef7bf412117548cb80b3f3b33a12977d,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,3299.344,380506.710,363546.990
14570,F99,5e3039a269ff005f2c76cea16c0cf783,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,773.460,55035.570,37949.550
14571,F99,5e3039a269ff005f2c76cea16c0cf783,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,319.296,25920.530,12437.100
14572,F99,6be0907d36fbbf5f7378f32c46f37943,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,2511.510,75197.620,56061.950


In [57]:
checks_with_offers_pred_action_pivot_price = checks_with_offers_pred_action.pivot_table(index=['Offer_ID', 'sku', 'region_name', 'Promo_type', 'start_date'], values=['selling_price_per_unit', 'supplier_price_per_unit'] , aggfunc=['mean'])
checks_with_offers_pred_action_pivot_price.columns = ['mean_selling_price_per_unit_without_action', 'mean_supplier_price_per_unit_without_action']
checks_with_offers_pred_action_pivot_price = checks_with_offers_pred_action_pivot_price.reset_index()
checks_with_offers_pred_action_pivot_price

,Offer_ID,sku,region_name,Promo_type,start_date,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action
0,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,77.996,29.990
1,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,74.865,28.818
2,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,82.490,30.930
3,10,376f806100fa4c1eb6c3878ead12631d,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,83.990,30.930
4,10,66dd48b798614b7d193a83777c8a0fdc,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,83.990,30.983
...,...,...,...,...,...,...,...
12704,F98,ef7bf412117548cb80b3f3b33a12977d,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,141.456,105.032
12705,F99,5e3039a269ff005f2c76cea16c0cf783,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,74.711,47.297
12706,F99,5e3039a269ff005f2c76cea16c0cf783,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,83.753,37.983
12707,F99,6be0907d36fbbf5f7378f32c46f37943,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,38.316,26.490


In [58]:
checks_with_offers_pred_action_pivot_amount = checks_with_offers_pred_action.pivot_table(index=['Offer_ID', 'sku', 'region_name', 'Promo_type', 'start_date'], values=['num_sales', 'supplier_price', 'selling_price'] , aggfunc=['sum'])
checks_with_offers_pred_action_pivot_amount.columns = ['num_sales_without_action', 'selling_price_without_action', 'supplier_price_without_action']
checks_with_offers_pred_action_pivot_amount = checks_with_offers_pred_action_pivot_amount.reset_index()
checks_with_offers_pred_action_pivot_amount

,Offer_ID,sku,region_name,Promo_type,start_date,num_sales_without_action,selling_price_without_action,supplier_price_without_action
0,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,20.000,1562.700,598.590
1,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,34.000,2537.990,986.540
2,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,2.000,164.980,61.860
3,10,376f806100fa4c1eb6c3878ead12631d,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,1.000,83.990,30.930
4,10,66dd48b798614b7d193a83777c8a0fdc,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,5.000,419.950,154.860
...,...,...,...,...,...,...,...,...
12704,F98,ef7bf412117548cb80b3f3b33a12977d,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,506.986,71652.860,53380.760
12705,F99,5e3039a269ff005f2c76cea16c0cf783,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,503.724,37428.130,23808.970
12706,F99,5e3039a269ff005f2c76cea16c0cf783,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,321.260,26918.890,12243.790
12707,F99,6be0907d36fbbf5f7378f32c46f37943,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,685.386,26440.340,18299.620


In [59]:
checks_pred_action_merged = checks_with_offers_pred_action_pivot_price.merge(checks_with_offers_pred_action_pivot_amount, how='outer', on = ['Offer_ID', 'sku', 'region_name', 'start_date'])
checks_pred_action_merged

,Offer_ID,sku,region_name,Promo_type_x,start_date,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action,Promo_type_y,num_sales_without_action,selling_price_without_action,supplier_price_without_action
0,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,77.996,29.990,Biweekly,20.000,1562.700,598.590
1,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,74.865,28.818,Biweekly,34.000,2537.990,986.540
2,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,82.490,30.930,Biweekly,2.000,164.980,61.860
3,10,376f806100fa4c1eb6c3878ead12631d,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,83.990,30.930,Biweekly,1.000,83.990,30.930
4,10,66dd48b798614b7d193a83777c8a0fdc,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,83.990,30.983,Biweekly,5.000,419.950,154.860
...,...,...,...,...,...,...,...,...,...,...,...
12704,F98,ef7bf412117548cb80b3f3b33a12977d,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,141.456,105.032,Facades,506.986,71652.860,53380.760
12705,F99,5e3039a269ff005f2c76cea16c0cf783,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,74.711,47.297,Facades,503.724,37428.130,23808.970
12706,F99,5e3039a269ff005f2c76cea16c0cf783,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,83.753,37.983,Facades,321.260,26918.890,12243.790
12707,F99,6be0907d36fbbf5f7378f32c46f37943,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,38.316,26.490,Facades,685.386,26440.340,18299.620


In [60]:
checks_with_action_merged = checks_with_offers_action_pivot_price.merge(checks_with_offers_action_pivot_amount, how='outer', on = ['Offer_ID', 'sku', 'region_name', 'start_date'])
checks_with_action_merged

,Offer_ID,sku,region_name,Promo_type_x,start_date,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action,Promo_type_y,num_sales_action,selling_price_action,supplier_price_action
0,10,172239f5f436f6b6661fdaa10fb47e9f,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,49.290,28.623,Biweekly,95.000,4684.180,2715.750
1,10,172239f5f436f6b6661fdaa10fb47e9f,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,48.832,28.504,Biweekly,103.000,5036.900,2936.440
2,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,49.355,28.832,Biweekly,289.000,14265.200,8327.790
3,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,49.372,28.428,Biweekly,521.000,25722.260,14813.040
4,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,49.346,31.137,Biweekly,111.000,5477.810,3457.240
...,...,...,...,...,...,...,...,...,...,...,...
14569,F98,ef7bf412117548cb80b3f3b33a12977d,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,115.438,110.175,Facades,3299.344,380506.710,363546.990
14570,F99,5e3039a269ff005f2c76cea16c0cf783,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,71.952,48.791,Facades,773.460,55035.570,37949.550
14571,F99,5e3039a269ff005f2c76cea16c0cf783,7e35e74e610188414ad24235dd787c78,Facades,2020-06-18,81.594,38.961,Facades,319.296,25920.530,12437.100
14572,F99,6be0907d36fbbf5f7378f32c46f37943,152f1b77a32508570e2745daf9ce7aec,Facades,2020-06-18,29.943,22.317,Facades,2511.510,75197.620,56061.950


In [61]:
checks_offers_merged = checks_with_action_merged.merge(checks_pred_action_merged, how='outer', on = ['Offer_ID', 'sku', 'region_name', 'start_date'])
checks_offers_merged

,Offer_ID,sku,region_name,Promo_type_x_x,start_date,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action,Promo_type_y_x,num_sales_action,selling_price_action,supplier_price_action,Promo_type_x_y,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action,Promo_type_y_y,num_sales_without_action,selling_price_without_action,supplier_price_without_action
0,10,172239f5f436f6b6661fdaa10fb47e9f,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,49.290,28.623,Biweekly,95.000,4684.180,2715.750,NaN,nan,nan,NaN,nan,nan,nan
1,10,172239f5f436f6b6661fdaa10fb47e9f,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,48.832,28.504,Biweekly,103.000,5036.900,2936.440,NaN,nan,nan,NaN,nan,nan,nan
2,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,49.355,28.832,Biweekly,289.000,14265.200,8327.790,Biweekly,77.996,29.990,Biweekly,20.000,1562.700,598.590
3,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,49.372,28.428,Biweekly,521.000,25722.260,14813.040,Biweekly,74.865,28.818,Biweekly,34.000,2537.990,986.540
4,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,49.346,31.137,Biweekly,111.000,5477.810,3457.240,Biweekly,82.490,30.930,Biweekly,2.000,164.980,61.860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14769,F5,edbe917ef00874205ad893c3931ea9e0,7e35e74e610188414ad24235dd787c78,NaN,2020-01-09,nan,nan,NaN,nan,nan,nan,Facades,101.990,95.600,Facades,1.000,101.990,95.600
14770,F50,f3f807ff0fe767b94827bc9d8367d324,152f1b77a32508570e2745daf9ce7aec,NaN,2020-03-26,nan,nan,NaN,nan,nan,nan,Facades,188.990,95.240,Facades,1.000,188.990,95.240
14771,F50,fe1d8a29a7ed76cd79f0a520ed18e0e6,152f1b77a32508570e2745daf9ce7aec,NaN,2020-03-26,nan,nan,NaN,nan,nan,nan,Facades,188.410,101.300,Facades,1.000,188.410,101.300
14772,F53,6b046ae5ddceb4ca2dca623fbc9011cf,7e35e74e610188414ad24235dd787c78,NaN,2020-04-02,nan,nan,NaN,nan,nan,nan,Facades,74.890,57.280,Facades,2.000,149.780,114.560


In [62]:
checks_offers_merged_without_null = checks_offers_merged.dropna()
checks_offers_merged_without_null.head()

,Offer_ID,sku,region_name,Promo_type_x_x,start_date,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action,Promo_type_y_x,num_sales_action,selling_price_action,supplier_price_action,Promo_type_x_y,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action,Promo_type_y_y,num_sales_without_action,selling_price_without_action,supplier_price_without_action
2,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,49.355,28.832,Biweekly,289.000,14265.200,8327.790,Biweekly,77.996,29.990,Biweekly,20.000,1562.700,598.590
3,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,49.372,28.428,Biweekly,521.000,25722.260,14813.040,Biweekly,74.865,28.818,Biweekly,34.000,2537.990,986.540
4,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,49.346,31.137,Biweekly,111.000,5477.810,3457.240,Biweekly,82.490,30.930,Biweekly,2.000,164.980,61.860
5,10,376f806100fa4c1eb6c3878ead12631d,7e35e74e610188414ad24235dd787c78,Biweekly,2020-04-02,49.387,31.173,Biweekly,174.000,8592.680,5419.670,Biweekly,83.990,30.930,Biweekly,1.000,83.990,30.930
6,10,66dd48b798614b7d193a83777c8a0fdc,152f1b77a32508570e2745daf9ce7aec,Biweekly,2020-04-02,49.369,31.012,Biweekly,144.000,7108.950,4464.550,Biweekly,83.990,30.983,Biweekly,5.000,419.950,154.860


In [63]:
checks_offers_merged_without_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12509 entries, 2 to 14573
Data columns (total 18 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   Offer_ID                                     12509 non-null  object        
 1   sku                                          12509 non-null  object        
 2   region_name                                  12509 non-null  object        
 3   Promo_type_x_x                               12509 non-null  object        
 4   start_date                                   12509 non-null  datetime64[ns]
 5   mean_selling_price_per_unit_action           12509 non-null  float64       
 6   mean_supplier_price_per_unit_action          12509 non-null  float64       
 7   Promo_type_y_x                               12509 non-null  object        
 8   num_sales_action                             12509 non-null  float64       


In [64]:
del checks_offers_merged_without_null['Promo_type_y_x']
del checks_offers_merged_without_null['Promo_type_y_y']

In [65]:
checks_offers_merged_without_null['margin_pred_action'] = checks_offers_merged_without_null['mean_selling_price_per_unit_without_action'] / checks_offers_merged_without_null['mean_supplier_price_per_unit_without_action'] - 1

In [66]:
checks_offers_merged_without_null['margin_action'] = checks_offers_merged_without_null['mean_selling_price_per_unit_action'] / checks_offers_merged_without_null['mean_supplier_price_per_unit_action'] - 1

In [67]:
checks_offers_merged_without_null['margin_delta'] = checks_offers_merged_without_null['margin_action'] / checks_offers_merged_without_null['margin_pred_action'] - 1

In [68]:
checks_offers_merged_without_null['discount'] = checks_offers_merged_without_null['mean_selling_price_per_unit_action'] / checks_offers_merged_without_null['mean_selling_price_per_unit_without_action'] - 1

In [93]:
checks_offers_merged_without_null['uplift'] = checks_offers_merged_without_null['selling_price_action'] / checks_offers_merged_without_null['selling_price_without_action'] - 1
checks_offers_merged_without_null

,Offer_ID,sku,region_name,Promo_type_x_x,start_date,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action,num_sales_action,selling_price_action,supplier_price_action,Promo_type_x_y,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action,num_sales_without_action,selling_price_without_action,supplier_price_without_action,margin_pred_action,margin_action,margin_delta,discount,uplift
11677,F146,e8d6d7ff8d26a1ae575e7ae15d2fac75,152f1b77a32508570e2745daf9ce7aec,Facades,2020-09-10,157.984,137.687,7.000,1105.890,963.810,Facades,168.810,136.785,8.000,1350.480,1094.280,0.234,0.147,-0.370,-0.064,-0.181
11633,F145,4c9844b76ae889bf8caf67423973ed03,152f1b77a32508570e2745daf9ce7aec,Facades,2020-09-10,37.490,17.920,3.000,112.470,53.760,Facades,53.710,17.920,1.000,53.710,17.920,1.997,1.092,-0.453,-0.302,1.094
11662,F146,726f47a2222121a111ee8ca1b0469cb8,7e35e74e610188414ad24235dd787c78,Facades,2020-09-10,157.773,154.460,6.000,946.640,926.760,Facades,168.990,153.798,4.000,675.960,615.190,0.099,0.021,-0.783,-0.066,0.400
11661,F146,726f47a2222121a111ee8ca1b0469cb8,152f1b77a32508570e2745daf9ce7aec,Facades,2020-09-10,157.830,138.501,8.000,1262.640,1108.010,Facades,168.620,137.020,1.000,168.620,137.020,0.231,0.140,-0.395,-0.064,6.488
11660,F146,70482e6d7da4064424c5c6eaa98e54d7,7e35e74e610188414ad24235dd787c78,Facades,2020-09-10,157.901,110.737,9.000,1421.110,996.630,Facades,168.837,152.710,3.000,506.510,458.130,0.106,0.426,3.033,-0.065,1.806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8194,652,30d696491aa04a7837155825be0575e2,7e35e74e610188414ad24235dd787c78,Seasonal,2019-09-24,799.685,474.152,20.000,15993.690,9483.030,Seasonal,1073.950,482.998,3.000,2947.400,1461.120,1.224,0.687,-0.439,-0.255,4.426
8195,652,c495c207866eb6b20f221ece1446719e,152f1b77a32508570e2745daf9ce7aec,Seasonal,2019-09-24,699.431,409.869,31.000,21683.240,12699.610,Seasonal,973.860,417.890,2.000,1947.720,835.780,1.330,0.706,-0.469,-0.282,10.133
8196,652,c495c207866eb6b20f221ece1446719e,7e35e74e610188414ad24235dd787c78,Seasonal,2019-09-24,699.249,408.065,19.000,13285.740,7753.230,Seasonal,1248.797,409.993,5.000,6243.780,2046.250,2.046,0.714,-0.651,-0.440,1.128
8198,652,da3906535e426d7aff1a1c2d48879449,7e35e74e610188414ad24235dd787c78,Seasonal,2019-09-24,399.507,209.864,69.000,27570.880,14479.760,Seasonal,399.990,209.720,1.000,399.990,209.720,0.907,0.904,-0.004,-0.001,67.929


In [94]:
checks_offers_merged_without_null = checks_offers_merged_without_null.sort_values(by = 'start_date', ascending = False)

In [109]:
checks_offers_merged_without_null = checks_offers_merged_without_null.merge(hierarchy, how='inner', on = ['sku'])
checks_offers_merged_without_null

,Offer_ID,sku,region_name,Promo_type_x_x,start_date,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action,num_sales_action,selling_price_action,supplier_price_action,Promo_type_x_y,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action,num_sales_without_action,selling_price_without_action,supplier_price_without_action,margin_pred_action,margin_action,margin_delta,discount,uplift,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4
0,F146,e8d6d7ff8d26a1ae575e7ae15d2fac75,152f1b77a32508570e2745daf9ce7aec,Facades,2020-09-10,157.984,137.687,7.000,1105.890,963.810,Facades,168.810,136.785,8.000,1350.480,1094.280,0.234,0.147,-0.370,-0.064,-0.181,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,3341f6f048384ec73a7ba2e77d2db48b,ee087215b0e0761bc986abd837ac7ab7
1,F146,e8d6d7ff8d26a1ae575e7ae15d2fac75,7e35e74e610188414ad24235dd787c78,Facades,2020-09-10,157.990,147.219,7.000,1105.930,1030.530,Facades,168.750,154.330,4.000,675.000,617.320,0.093,0.073,-0.217,-0.064,0.638,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,3341f6f048384ec73a7ba2e77d2db48b,ee087215b0e0761bc986abd837ac7ab7
2,F132,e8d6d7ff8d26a1ae575e7ae15d2fac75,152f1b77a32508570e2745daf9ce7aec,Facades,2020-08-20,157.784,145.605,14.000,2208.980,2038.470,Facades,168.942,153.674,5.000,844.710,768.370,0.099,0.084,-0.158,-0.066,1.615,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,3341f6f048384ec73a7ba2e77d2db48b,ee087215b0e0761bc986abd837ac7ab7
3,F132,e8d6d7ff8d26a1ae575e7ae15d2fac75,7e35e74e610188414ad24235dd787c78,Facades,2020-08-20,157.936,129.698,9.000,1421.420,1167.280,Facades,168.838,189.024,5.000,844.190,945.120,-0.107,0.218,-3.039,-0.065,0.684,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,3341f6f048384ec73a7ba2e77d2db48b,ee087215b0e0761bc986abd837ac7ab7
4,F105,e8d6d7ff8d26a1ae575e7ae15d2fac75,7e35e74e610188414ad24235dd787c78,Facades,2020-07-02,168.808,140.442,5.000,844.040,702.210,Facades,207.557,140.633,3.000,622.670,421.900,0.476,0.202,-0.576,-0.187,0.356,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,3341f6f048384ec73a7ba2e77d2db48b,ee087215b0e0761bc986abd837ac7ab7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12504,635,98bc1225070a118d060eb5ac021ff036,152f1b77a32508570e2745daf9ce7aec,Seasonal,2019-09-24,799.000,470.090,2.000,1598.000,940.180,Seasonal,1598.280,470.090,1.000,1598.280,470.090,2.400,0.700,-0.708,-0.500,-0.000,44a7b825383402ed937a15fb8a5bb12a,2838023a778dfaecdc212708f721b788,48f7d3043bc03e6c48a6f0ebc0f258a8,55fbc2980e7b8868bd61c9b13d81915f
12505,638,cef10d26373f6ec91fed1cef6573a293,7e35e74e610188414ad24235dd787c78,Seasonal,2019-09-24,287.317,213.564,20.000,5757.600,4271.280,Seasonal,298.605,213.560,2.000,597.210,427.120,0.398,0.345,-0.133,-0.038,8.641,44a7b825383402ed937a15fb8a5bb12a,a684eceee76fc522773286a895bc8436,c4098249cc953bd98b54e57a1ad89d12,c38b6b06d1e06be139a52a902533909b
12506,638,cef10d26373f6ec91fed1cef6573a293,152f1b77a32508570e2745daf9ce7aec,Seasonal,2019-09-24,288.543,213.577,44.000,12601.970,9397.630,Seasonal,298.940,213.560,3.000,896.760,640.680,0.400,0.351,-0.122,-0.035,13.053,44a7b825383402ed937a15fb8a5bb12a,a684eceee76fc522773286a895bc8436,c4098249cc953bd98b54e57a1ad89d12,c38b6b06d1e06be139a52a902533909b
12507,636,f37e393f23bc2e5fc27c583ec021783b,7e35e74e610188414ad24235dd787c78,Seasonal,2019-09-24,298.892,246.000,12.000,3586.810,2952.000,Seasonal,599.000,246.000,1.000,599.000,246.000,1.435,0.215,-0.850,-0.501,4.988,44a7b825383402ed937a15fb8a5bb12a,2838023a778dfaecdc212708f721b788,4eff0720836a198b6174eecf02cbfdbf,bb2661f40476b4b87e1aee31a3ef6fb8


In [110]:
date_devide = '2020-06-01'

In [111]:
train = checks_offers_merged_without_null.query('start_date <= @date_devide')
train

,Offer_ID,sku,region_name,Promo_type_x_x,start_date,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action,num_sales_action,selling_price_action,supplier_price_action,Promo_type_x_y,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action,num_sales_without_action,selling_price_without_action,supplier_price_without_action,margin_pred_action,margin_action,margin_delta,discount,uplift,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4
6,F85,e8d6d7ff8d26a1ae575e7ae15d2fac75,152f1b77a32508570e2745daf9ce7aec,Facades,2020-05-28,168.784,139.861,16.000,2700.540,2237.770,Facades,164.012,138.370,6.000,984.070,830.220,0.185,0.207,0.116,0.029,1.744,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,3341f6f048384ec73a7ba2e77d2db48b,ee087215b0e0761bc986abd837ac7ab7
7,F85,e8d6d7ff8d26a1ae575e7ae15d2fac75,7e35e74e610188414ad24235dd787c78,Facades,2020-05-28,168.859,143.328,10.000,1688.590,1433.280,Facades,168.931,145.816,7.000,1182.520,1020.710,0.159,0.178,0.124,-0.000,0.428,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,3341f6f048384ec73a7ba2e77d2db48b,ee087215b0e0761bc986abd837ac7ab7
8,F50,e8d6d7ff8d26a1ae575e7ae15d2fac75,7e35e74e610188414ad24235dd787c78,Facades,2020-03-26,158.921,161.594,22.000,3496.460,3549.520,Facades,234.945,184.746,5.000,1216.470,917.150,0.272,-0.017,-1.061,-0.324,1.874,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,3341f6f048384ec73a7ba2e77d2db48b,ee087215b0e0761bc986abd837ac7ab7
9,F50,e8d6d7ff8d26a1ae575e7ae15d2fac75,152f1b77a32508570e2745daf9ce7aec,Facades,2020-03-26,158.839,137.951,16.000,2541.430,2207.210,Facades,188.990,135.820,3.000,566.970,407.460,0.391,0.151,-0.613,-0.160,3.482,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,3341f6f048384ec73a7ba2e77d2db48b,ee087215b0e0761bc986abd837ac7ab7
10,597,e8d6d7ff8d26a1ae575e7ae15d2fac75,7e35e74e610188414ad24235dd787c78,Biweekly,2019-11-28,154.069,139.028,35.000,5393.340,4863.440,Biweekly,157.667,138.924,27.000,4257.470,3750.950,0.135,0.108,-0.198,-0.023,0.267,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,3341f6f048384ec73a7ba2e77d2db48b,ee087215b0e0761bc986abd837ac7ab7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12504,635,98bc1225070a118d060eb5ac021ff036,152f1b77a32508570e2745daf9ce7aec,Seasonal,2019-09-24,799.000,470.090,2.000,1598.000,940.180,Seasonal,1598.280,470.090,1.000,1598.280,470.090,2.400,0.700,-0.708,-0.500,-0.000,44a7b825383402ed937a15fb8a5bb12a,2838023a778dfaecdc212708f721b788,48f7d3043bc03e6c48a6f0ebc0f258a8,55fbc2980e7b8868bd61c9b13d81915f
12505,638,cef10d26373f6ec91fed1cef6573a293,7e35e74e610188414ad24235dd787c78,Seasonal,2019-09-24,287.317,213.564,20.000,5757.600,4271.280,Seasonal,298.605,213.560,2.000,597.210,427.120,0.398,0.345,-0.133,-0.038,8.641,44a7b825383402ed937a15fb8a5bb12a,a684eceee76fc522773286a895bc8436,c4098249cc953bd98b54e57a1ad89d12,c38b6b06d1e06be139a52a902533909b
12506,638,cef10d26373f6ec91fed1cef6573a293,152f1b77a32508570e2745daf9ce7aec,Seasonal,2019-09-24,288.543,213.577,44.000,12601.970,9397.630,Seasonal,298.940,213.560,3.000,896.760,640.680,0.400,0.351,-0.122,-0.035,13.053,44a7b825383402ed937a15fb8a5bb12a,a684eceee76fc522773286a895bc8436,c4098249cc953bd98b54e57a1ad89d12,c38b6b06d1e06be139a52a902533909b
12507,636,f37e393f23bc2e5fc27c583ec021783b,7e35e74e610188414ad24235dd787c78,Seasonal,2019-09-24,298.892,246.000,12.000,3586.810,2952.000,Seasonal,599.000,246.000,1.000,599.000,246.000,1.435,0.215,-0.850,-0.501,4.988,44a7b825383402ed937a15fb8a5bb12a,2838023a778dfaecdc212708f721b788,4eff0720836a198b6174eecf02cbfdbf,bb2661f40476b4b87e1aee31a3ef6fb8


In [112]:
test = checks_offers_merged_without_null.query('start_date > @date_devide')
test

,Offer_ID,sku,region_name,Promo_type_x_x,start_date,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action,num_sales_action,selling_price_action,supplier_price_action,Promo_type_x_y,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action,num_sales_without_action,selling_price_without_action,supplier_price_without_action,margin_pred_action,margin_action,margin_delta,discount,uplift,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4
0,F146,e8d6d7ff8d26a1ae575e7ae15d2fac75,152f1b77a32508570e2745daf9ce7aec,Facades,2020-09-10,157.984,137.687,7.000,1105.890,963.810,Facades,168.810,136.785,8.000,1350.480,1094.280,0.234,0.147,-0.370,-0.064,-0.181,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,3341f6f048384ec73a7ba2e77d2db48b,ee087215b0e0761bc986abd837ac7ab7
1,F146,e8d6d7ff8d26a1ae575e7ae15d2fac75,7e35e74e610188414ad24235dd787c78,Facades,2020-09-10,157.990,147.219,7.000,1105.930,1030.530,Facades,168.750,154.330,4.000,675.000,617.320,0.093,0.073,-0.217,-0.064,0.638,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,3341f6f048384ec73a7ba2e77d2db48b,ee087215b0e0761bc986abd837ac7ab7
2,F132,e8d6d7ff8d26a1ae575e7ae15d2fac75,152f1b77a32508570e2745daf9ce7aec,Facades,2020-08-20,157.784,145.605,14.000,2208.980,2038.470,Facades,168.942,153.674,5.000,844.710,768.370,0.099,0.084,-0.158,-0.066,1.615,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,3341f6f048384ec73a7ba2e77d2db48b,ee087215b0e0761bc986abd837ac7ab7
3,F132,e8d6d7ff8d26a1ae575e7ae15d2fac75,7e35e74e610188414ad24235dd787c78,Facades,2020-08-20,157.936,129.698,9.000,1421.420,1167.280,Facades,168.838,189.024,5.000,844.190,945.120,-0.107,0.218,-3.039,-0.065,0.684,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,3341f6f048384ec73a7ba2e77d2db48b,ee087215b0e0761bc986abd837ac7ab7
4,F105,e8d6d7ff8d26a1ae575e7ae15d2fac75,7e35e74e610188414ad24235dd787c78,Facades,2020-07-02,168.808,140.442,5.000,844.040,702.210,Facades,207.557,140.633,3.000,622.670,421.900,0.476,0.202,-0.576,-0.187,0.356,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,3341f6f048384ec73a7ba2e77d2db48b,ee087215b0e0761bc986abd837ac7ab7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8063,22,25c8f8222aa5a4cd9e255e802de8446c,152f1b77a32508570e2745daf9ce7aec,Seasonal,2020-06-02,125.942,118.992,48.000,6045.160,5713.850,Seasonal,138.644,122.806,10.000,1374.790,1226.100,0.129,0.058,-0.547,-0.092,3.397,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,9d949c3d8baa0f9df6f22c4661946a61,e639633851413be3d5159f8c9712e039
8072,22,1bbb4afd56023eed74ec58c11fd20801,7e35e74e610188414ad24235dd787c78,Seasonal,2020-06-02,148.711,144.864,33.000,4907.400,4783.960,Seasonal,176.273,146.360,13.000,2265.270,1897.480,0.204,0.027,-0.870,-0.156,1.166,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,c09f9caf5e08836d4673ccdd69bb041e,0f2a4dac2d2eab08974187841e5664e7
8073,22,1bbb4afd56023eed74ec58c11fd20801,152f1b77a32508570e2745daf9ce7aec,Seasonal,2020-06-02,148.973,142.678,13.000,1936.650,1854.820,Seasonal,162.503,146.636,10.000,1600.000,1475.960,0.108,0.044,-0.592,-0.083,0.210,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,c09f9caf5e08836d4673ccdd69bb041e,0f2a4dac2d2eab08974187841e5664e7
8082,31,70e44df06e309c39b17dea56618b75af,7e35e74e610188414ad24235dd787c78,Seasonal,2020-06-02,26.609,17.187,1894.000,47158.350,32516.160,Seasonal,27.866,17.591,1027.000,27397.390,18038.920,0.584,0.548,-0.061,-0.045,0.721,cc9284597026dcc021735c0970ca0c93,c16a5320fa475530d9583c34fd356ef5,4aec1b3435c52abbdf8334ea0e7141e0,8a2e14258355535593b888f82a1b9a08


## Моделирование

## Модель CatBoost

In [113]:
train.columns

Index(['Offer_ID', 'sku', 'region_name', 'Promo_type_x_x', 'start_date',
       'mean_selling_price_per_unit_action',
       'mean_supplier_price_per_unit_action', 'num_sales_action',
       'selling_price_action', 'supplier_price_action', 'Promo_type_x_y',
       'mean_selling_price_per_unit_without_action',
       'mean_supplier_price_per_unit_without_action',
       'num_sales_without_action', 'selling_price_without_action',
       'supplier_price_without_action', 'margin_pred_action', 'margin_action',
       'margin_delta', 'discount', 'uplift', 'hierarchy_level1',
       'hierarchy_level2', 'hierarchy_level3', 'hierarchy_level4'],
      dtype='object')

In [115]:
features_train = train[['Offer_ID', 'sku', 'region_name', 'Promo_type_x_x', 'num_sales_without_action', 'margin_delta', 'discount', 'hierarchy_level1','hierarchy_level2', 'hierarchy_level3', 'hierarchy_level4']]
target_train = train['uplift']
features_test = test[['Offer_ID', 'sku', 'region_name', 'Promo_type_x_x', 'num_sales_without_action', 'margin_delta', 'discount', 'hierarchy_level1','hierarchy_level2', 'hierarchy_level3', 'hierarchy_level4']]
target_test = test['uplift']

In [117]:
cat_feature = ['Offer_ID', 'sku', 'region_name', 'Promo_type_x_x', 'hierarchy_level1','hierarchy_level2', 'hierarchy_level3', 'hierarchy_level4']

In [118]:
model_cat = CatBoostRegressor(loss_function='MAE', random_seed=12345, iterations=150)
model_cat.fit(features_train, target_train, cat_features=cat_feature, verbose=10)

0:	learn: 4.3814405	total: 5.14ms	remaining: 766ms
10:	learn: 3.9601419	total: 33.3ms	remaining: 420ms
20:	learn: 3.6628413	total: 62.9ms	remaining: 387ms
30:	learn: 3.4770178	total: 90.6ms	remaining: 348ms
40:	learn: 3.3539413	total: 118ms	remaining: 314ms
50:	learn: 3.2554767	total: 146ms	remaining: 284ms
60:	learn: 3.1827801	total: 175ms	remaining: 255ms
70:	learn: 3.1284208	total: 204ms	remaining: 227ms
80:	learn: 3.0762333	total: 232ms	remaining: 198ms
90:	learn: 3.0307853	total: 259ms	remaining: 168ms
100:	learn: 2.9908955	total: 287ms	remaining: 139ms
110:	learn: 2.9668603	total: 314ms	remaining: 110ms
120:	learn: 2.9413479	total: 342ms	remaining: 81.9ms
130:	learn: 2.9136994	total: 369ms	remaining: 53.5ms
140:	learn: 2.8921791	total: 395ms	remaining: 25.2ms
149:	learn: 2.8732367	total: 421ms	remaining: 0us


In [119]:
predictions_cat_train = model_cat.predict(features_train)
predictions_cat_test = model_cat.predict(features_test)

In [120]:
result_cat_train = mean_absolute_error(target_train, predictions_cat_train)
result_cat_train

2.6991081903931704

In [121]:
result_cat_test = mean_absolute_error(target_test, predictions_cat_test)
print("MAE для catboost:", result_cat_test)

MAE для catboost: 3.1927006891275513


In [122]:
r2_cat = r2_score(target_train, predictions_cat_train)
r2_cat

0.4770537191600791

In [123]:
r2_cat = r2_score(target_test, predictions_cat_test)
r2_cat

0.39443556328952756

## Модель LightGMB

In [124]:
features_train['Offer_ID'] = features_train['Offer_ID'].astype('category')
features_test['Offer_ID'] = features_test['Offer_ID'].astype('category')
features_train['sku'] = features_train['sku'].astype('category')
features_test['sku'] = features_test['sku'].astype('category')
features_train['region_name'] = features_train['region_name'].astype('category')
features_test['region_name'] = features_test['region_name'].astype('category')
features_train['Promo_type_x_x'] = features_train['Promo_type_x_x'].astype('category')
features_test['Promo_type_x_x'] = features_test['Promo_type_x_x'].astype('category')
features_train['hierarchy_level1'] = features_train['hierarchy_level1'].astype('category')
features_test['hierarchy_level1'] = features_test['hierarchy_level1'].astype('category')
features_train['hierarchy_level2'] = features_train['hierarchy_level2'].astype('category')
features_test['hierarchy_level2'] = features_test['hierarchy_level2'].astype('category')
features_train['hierarchy_level3'] = features_train['hierarchy_level3'].astype('category')
features_test['hierarchy_level3'] = features_test['hierarchy_level3'].astype('category')
features_train['hierarchy_level4'] = features_train['hierarchy_level4'].astype('category')
features_test['hierarchy_level4'] = features_test['hierarchy_level4'].astype('category')

In [125]:
cat_feature = ['Offer_ID', 'sku', 'region_name', 'Promo_type_x_x', 'hierarchy_level1','hierarchy_level2', 'hierarchy_level3', 'hierarchy_level4']
model_light = LGBMRegressor(num_leaves = 31, max_depth = -1, random_state=12345, n_estimators=100, categorical_feature = cat_feature)
model_light.fit(features_train, target_train, verbose=10)

LGBMRegressor(boosting_type='gbdt',
              categorical_feature=['Offer_ID', 'sku', 'region_name',
                                   'Promo_type_x_x', 'hierarchy_level1',
                                   'hierarchy_level2', 'hierarchy_level3',
                                   'hierarchy_level4'],
              class_weight=None, colsample_bytree=1.0, importance_type='split',
              learning_rate=0.1, max_depth=-1, min_child_samples=20,
              min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
              n_jobs=-1, num_leaves=31, objective=None, random_state=12345,
              reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [126]:
predictions_light_train = model_light.predict(features_train)

In [127]:
result_light = mean_absolute_error(target_train, predictions_light_train)
print("MAE для lightgmb:", result_light)

MAE для lightgmb: 1.9229280081012654


In [128]:
predictions_light_test = model_light.predict(features_test)

In [129]:
result_light = mean_absolute_error(target_test, predictions_light_test)
print("MAE для lightgmb:", result_light)

MAE для lightgmb: 3.2675808445218815
